In [1]:
%cd ..

/home/eli/AnacondaProjects/PassiveMNIST


In [2]:
import cv2 as cv
import datetime
import numpy as np
import os
import shutil

In [3]:
FPS = 120
DISTANCE_TO_SCREEN = 106
# SCALE_MOVIE = 15
SCALE_MOVIE = (1920 / 128, 1080 / 72)
SCREEN_DIMS = (100, 62)
SCREEN_RES = (1920, 1080)
SCALED_VIDEO_RES = (SCREEN_RES[0], SCREEN_RES[1])
VIDEO_SECONDS = 10
VIDEO_RES = (128, 72)
VIDEO_THRESHOLDS = [
    [(SCREEN_RES[0] - SCALED_VIDEO_RES[0]) / 2, (SCREEN_RES[1] - SCALED_VIDEO_RES[1]) / 2],
    [(SCREEN_RES[0] + SCALED_VIDEO_RES[0]) / 2, (SCREEN_RES[1] + SCALED_VIDEO_RES[1]) / 2]
]

In [4]:
SCREEN_HALFWIDTH_DEGREES = np.degrees(np.arctan(SCREEN_DIMS[0] / 2 / DISTANCE_TO_SCREEN))
SCREEN_HALFHEIGHT_DEGREES = np.degrees(np.arctan(SCREEN_DIMS[1] / 2 / DISTANCE_TO_SCREEN))

In [5]:
def degrees_to_pixels(theta):
    return theta * SCREEN_RES[0] / (2 * SCREEN_HALFWIDTH_DEGREES)

In [6]:
# LOWER PIXEL X => LEFTER
# LOWER PIXEL Y => HIGHER
def circle_to_pixels(x, y, r):
    x = SCREEN_RES[0] / 2 + degrees_to_pixels(x)
    y = SCREEN_RES[1] / 2 - degrees_to_pixels(y)
    return float(x), float(y), float(degrees_to_pixels(r))

In [7]:
HOLD_DEGREES = 3

In [14]:
def rfs_to_pixels(rfs):
    for ((x, y, r), p) in rfs:
        x, y, r = circle_to_pixels(x, y, r + HOLD_DEGREES)
        x = (x - VIDEO_THRESHOLDS[0][0]) / SCALE_MOVIE[0]
        y = (y - VIDEO_THRESHOLDS[0][1]) / SCALE_MOVIE[1]
        r /= np.array(SCALE_MOVIE).mean()
        if x >= 0 and x < VIDEO_RES[0] and y >= 0 and y < VIDEO_RES[1]:
            yield ((round(x), round(y), round(r)), p)

In [9]:
RFS = [(1.7654, -4.1565, 3.8086)]

In [10]:
from PassiveMNIST import sim_bmnist

In [11]:
NUM_VIDEOS = 5
TIMESTEPS = VIDEO_SECONDS * FPS

In [12]:
for d in range(1, 4):
    simulator = sim_bmnist.Sim_BMNIST(TIMESTEPS, d, VIDEO_RES, 0.015, 10)
    dirpath = 'data/movmnist_%d/' % d
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)
    shutil.copy('data/train-images-idx3-ubyte.gz', dirpath)
    simulator.sim_save_data(NUM_VIDEOS, dirpath)

    bmnists = np.load(dirpath + "ob-1.npy")
    for v in range(NUM_VIDEOS):
        video = (bmnists[v] * 255).astype('uint8')
        writer = cv.VideoWriter(dirpath + 'moving_mnist%d.mp4' % v, cv.VideoWriter_fourcc(*"mp4v"), FPS, SCREEN_RES, True)
        for t in range(bmnists[v].shape[0]):
            frame = cv.resize(video[t], SCREEN_RES)
            frame = np.repeat(frame[:, :, None], 3, axis=-1)
            writer.write(frame)
        writer.release()

Start to simulate bouncing mnist sequences...
(0s) Simulated 5 sequences, saved to 'data/movmnist_1/ob-1', 0 sequences left.
Start to simulate bouncing mnist sequences...
(2s) Simulated 5 sequences, saved to 'data/movmnist_2/ob-1', 0 sequences left.
Start to simulate bouncing mnist sequences...
(2s) Simulated 5 sequences, saved to 'data/movmnist_3/ob-1', 0 sequences left.
